In [1]:
import os, sys
sys.path.append("../../")

from src.core.module import Module, Linear, LayerNorm
from src.core.losses import CrossEntropy, BCE
from src.core.optim import Standard, AdamW
from src.core.tensor import Tensor
import numpy as np
import time
from typing import List
from src.tokenizer.tokenizer import Tokenizer
import pandas as pd

In [2]:
class Net(Module):
    def __init__(self):
        super().__init__()
        self.fc1 = self.linear(7, 10, name="fc1")
        self.fc2 = self.linear(10,1, name="fc2")
        self.ln = self.layer_norm(axis=-1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.dropout(x, p=0.1)
        x = self.ln(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
    
    def train(self, x: Tensor, y: Tensor, optimizer, num_epochs=10000):
        for epoch in range(num_epochs):
            y_hat = self.forward(x)
            
            loss = BCE(y_hat, y)
            
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            if epoch % 1 == 0:

                print(f"Epoch {epoch}, Loss: {loss.data}")

if __name__ == "__main__":
    df = pd.read_csv("../../src/experiments/data.csv")
    df['Quality'] = df['Quality'].apply(lambda x: 1 if x == "Good" else 0)
    X = Tensor(np.array(df.drop('Quality', axis=1).values))[:128]
    y = Tensor(np.array(df['Quality'].values).reshape((-1, 1)))[:128]

    X_test = Tensor(np.array(df.drop('Quality', axis=1).values))[128:]
    y_test = Tensor(np.array(df['Quality'].values).reshape((-1, 1)))[128:]

    net = Net()

    net._build(X.shape)
    optimizer = AdamW(net.parameters(), lr=0.001, clip_norm=10.0)

    net.train(X, y, optimizer)

    print(net.pipeline)


linear_0_linear_weight
linear_0_linear_bias
linear_0_linear_weight
linear_0_linear_bias
linear_0_layer_norm_gamma
linear_0_layer_norm_beta
Epoch 0, Loss: 0.6921524182510738
Epoch 1, Loss: 0.6882014843346098
Epoch 2, Loss: 0.6853309949388939
Epoch 3, Loss: 0.6814799679289711
Epoch 4, Loss: 0.6787607101451818
Epoch 5, Loss: 0.6753998610569996
Epoch 6, Loss: 0.6724472306779474
Epoch 7, Loss: 0.6696421307948859
Epoch 8, Loss: 0.6665615229305961
Epoch 9, Loss: 0.6637818878693487
Epoch 10, Loss: 0.6596269797708463
Epoch 11, Loss: 0.6564189884323874
Epoch 12, Loss: 0.6533826185130758
Epoch 13, Loss: 0.6502956265930373
Epoch 14, Loss: 0.6458489293759316
Epoch 15, Loss: 0.643993113770736
Epoch 16, Loss: 0.6403283476495363
Epoch 17, Loss: 0.6372505410215018
Epoch 18, Loss: 0.6355430003231965
Epoch 19, Loss: 0.6315690154278331
Epoch 20, Loss: 0.6283701064282452
Epoch 21, Loss: 0.625644555787374
Epoch 22, Loss: 0.6214142242231879
Epoch 23, Loss: 0.6188759576714368
Epoch 24, Loss: 0.614773470234810